In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date


In [15]:
result = Path('result/')
report = Path('report/')
instances = Path('../../data/csifa')

In [16]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = [0]*N
	PP  = [0]*N

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [ ]:
MAX_CPU_TIME = 60.0
EPSILON = 1e-6

def clsr_std_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C):
	try:

		# Create a new model
		model = gp.Model("clspr_std")

		# Create variables
		xp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xp")
		yp = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yp")
		sp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sp")
		xr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xr")
		yr = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yr")
		sr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sr")
		
		model.update()

		# set objective
		fobj = gp.quicksum(PP[i]*xp[i] for i in range(N))
		fobj += gp.quicksum(HP[i]*sp[i] for i in range(N))
		fobj += gp.quicksum(FP[i]*yp[i] for i in range(N))
		fobj += gp.quicksum(PR[i]*xr[i] for i in range(N))
		fobj += gp.quicksum(HR[i]*sr[i] for i in range(N))
		fobj += gp.quicksum(FR[i]*yr[i] for i in range(N))
	
		model.setObjective(fobj, sense = GRB.MINIMIZE)

		# add constraints
		model.addConstr(xp[0] + xr[0] - sp[0] == D[0])
		model.addConstrs(sp[i-1] + xp[i] + xr[i] - sp[i] == D[i] for i in range(N) if i > 0 )
		model.addConstr(R[0] - xr[0] - sr[0] == 0)
		model.addConstrs(sr[i-1] + R[i] - xr[i] - sr[i] == 0 for i in range(N) if i > 0)
		model.addConstrs(xp[i] - yp[i]*min(SD[i][N-1],C) <= 0 for i in range(N))
		model.addConstrs(xr[i] - yr[i]*min(SR[0][i],SD[i][N-1],C) <= 0 for i in range(N))
		model.addConstrs(xp[i] + xr[i] <= C for i in range(N))
	    
		# export .lp .mps
		#model.write(f"file_format/{file_name}+"_model.lp")

		# parameters 
		model.setParam("LogFile", "gurobi_log.log")
		model.setParam("LogToConsole", 0)

		#model.Params.LogFile("gurobi_log.log")
		model.Params.TimeLimit = MAX_CPU_TIME
		model.Params.MIPGap = EPSILON
		model.Params.Threads = 1
		model.Params.Cuts = -1
		model.Params.Presolve = -1
		#model.Params.OutputFlag = -1		
		model.Params.method = 0 #-1=automatic, 0=primal, 1=dual , 2=barrier
		model.Params.NodeMethod = 1 #-1=automatic, 0=primal, 1=dual , 2=barrier

		# optimize model
		model.optimize()
		
		tmp = 0
		if model.status == GRB.OPTIMAL:
			tmp = 1

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))
	
	objval = model.ObjVal
	objbound = model.ObjBound
	mipgap = model.MIPGap
	runtime = model.Runtime
	nodecount = model.NodeCount

	return objval, objbound, mipgap, runtime, nodecount, tmp

In [33]:
def main(file_name):
	
	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()

	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	bestsol, bestbound, gap, runtime, numnode, tmp = clsr_std_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C)
	
	arquivo = open(os.path.join(result,'clsr_std_mip.csv'),'a')
	
	arquivo.write(file_name+';'
			   +str(round(bestsol,2))+';'
			   +str(round(bestbound,2))+';'
			   +str(round(gap,2))+';'
			   +str(round(runtime,2))+';'
			   +str(round(numnode,2))+';'
			   +str(round(tmp,2))+'\n')
	arquivo.close()

In [34]:
if __name__== "__main__" :

	for dim in [52]:
		for id in range(1,2):
			datafile = f"c{dim}_{id}.txt"
			print(f"Resolvendo instancia {datafile}")			
			main(datafile)

Resolvendo instancia c52_1.txt
Set parameter LogFile to value "gurobi_log.log"
Set parameter LogToConsole to value 0


## Table of results

In [9]:
data=f'result/clsr_std_mip.csv'

df = pd.DataFrame()
df = pd.read_csv(data,header=None,sep=';')

tab = pd.DataFrame()
tab = pd.concat([tab, df], ignore_index=True)
tab.columns = ['instance','objval','objbound','mipgap','time','nodes','opt']

resume = pd.DataFrame({
    'instance':f"resume",
    'objval':tab["objval"].mean(),
    'objbound':tab["objbound"].mean(),
    'mipgap':tab['mipgap'].mean(),
    'time':tab['time'].mean(),
    'nodes':tab['nodes'].mean(),
    'opt':tab['opt'].sum(),
     },index=[f"uls_mip"]
)

tab = pd.concat([tab, resume], ignore_index=True)

tab["objval"] = tab["objval"].round(2)
tab["objbound"] = tab["objbound"].round(2)
tab["mipgap"] = tab["mipgap"].round(2)
tab["time"] = tab["time"].round(2)
tab["nodes"] = tab["nodes"].round(2)
tab["opt"] = tab["opt"].round().astype('Int64')



In [10]:
tab

,instance,objval,objbound,mipgap,time,nodes,opt
0,c52_1.txt,9768.80,9626.26,0.01,60.0,41598.0,0
1,c52_2.txt,10144.60,9916.57,0.02,60.0,53619.0,0
2,c52_3.txt,9347.80,9297.25,0.01,60.0,60169.0,0
3,c52_4.txt,10355.40,10227.52,0.01,60.0,62989.0,0
4,c52_5.txt,11111.50,11062.62,0.00,60.0,77239.0,0
5,resume,10145.62,10026.04,0.01,60.0,59122.8,0


In [11]:
#tab
print(
    tab[['instance','objval','objbound','mipgap','time','nodes','opt']].
    to_latex(index=False,float_format="%.2f")
)

\begin{tabular}{lrrrrrr}
\toprule
instance & objval & objbound & mipgap & time & nodes & opt \\
\midrule
c52_1.txt & 9768.80 & 9626.26 & 0.01 & 60.00 & 41598.00 & 0 \\
c52_2.txt & 10144.60 & 9916.57 & 0.02 & 60.00 & 53619.00 & 0 \\
c52_3.txt & 9347.80 & 9297.25 & 0.01 & 60.00 & 60169.00 & 0 \\
c52_4.txt & 10355.40 & 10227.52 & 0.01 & 60.00 & 62989.00 & 0 \\
c52_5.txt & 11111.50 & 11062.62 & 0.00 & 60.00 & 77239.00 & 0 \\
resume & 10145.62 & 10026.04 & 0.01 & 60.00 & 59122.80 & 0 \\
\bottomrule
\end{tabular}

